In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.compose
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import classification_report,f1_score,precision_score,recall_score,roc_auc_score
from sklearn.model_selection import GridSearchCV

In [2]:
def load_data(path, partition=0.3):
  # permite cargar train, validation y test data. 
    if path in ('train', 'validation'):
        df = pd.read_csv('https://raw.githubusercontent.com/gonzalomdvc/ml-datasets/master/train.csv')
        train, valid = train_test_split(df, test_size=partition)
    if path == 'train':
        x = train.drop(columns=['Credit_Score'])
        y = train['Credit_Score']
        return x, y
    elif path == 'validation':
        x = valid.drop(columns=['Credit_Score'])
        y = valid['Credit_Score']
        return x, y
    elif path == 'test':
        test = pd.read_csv('https://raw.githubusercontent.com/gonzalomdvc/ml-datasets/master/test.csv')
        return test

#Preprocesamiento

In [3]:
#Type_of_Loan(13) la dropeo, pero se puede hacer un oneHotEncoderManual
drop_columns = [0,1,3,5,13]
column_transformerX = sklearn.compose.ColumnTransformer(transformers=[    
    ("drop", "drop" , [0,1,3,5,13]),  #borramos todas las columnas que no nos aportan ninguna información 
    ("Month", sklearn.preprocessing.StandardScaler() , [2]),  
    ("Age", sklearn.preprocessing.StandardScaler() , [4]),  
    ("Trabajo", sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore') , [6]),#El handle_unknown='ignore' hace que se omita la asignacion de un elemento al encontrar un valor desconocido  
    ("IngresosAnuales", sklearn.preprocessing.StandardScaler() , [7]),  
    ("SalarioMensual", sklearn.preprocessing.StandardScaler() , [8]),  
    ("Num_Bank_Accounts", sklearn.preprocessing.StandardScaler() , [9]),  
    ("Num_Credit_Card", sklearn.preprocessing.StandardScaler() , [10]),  
    ("Interest_Rate", sklearn.preprocessing.StandardScaler() , [11]),  
    ("Num_of_Loan", sklearn.preprocessing.StandardScaler() , [12]),  
    ("Delay_from_due_date", sklearn.preprocessing.StandardScaler() , [14]),  
    ("Num_of_Delayed_Payment", sklearn.preprocessing.StandardScaler() , [15]),  
    ("Changed_Credit_Limit", sklearn.preprocessing.StandardScaler() , [16]),  
    ("Num_Credit_Inquiries", sklearn.preprocessing.StandardScaler() , [17]),  
    ("Credit_Mix", sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore') , [18]),
    ("Outstanding_Debt", sklearn.preprocessing.StandardScaler() , [19]),  
    ("Credit_Utilization_Ratio", sklearn.preprocessing.StandardScaler() , [20]),  
    ("Credit_History_Age", sklearn.preprocessing.StandardScaler() , [21]),  
    ("Payment_of_Min_Amount", sklearn.preprocessing.OneHotEncoder() , [22]),
    ("Total_EMI_per_month", sklearn.preprocessing.StandardScaler() , [23]),  
    ("Amount_invested_monthly", sklearn.preprocessing.StandardScaler() , [24]),  
    ("Payment_Behaviour", sklearn.preprocessing.OneHotEncoder() , [25]),  
    ("Monthly_Balance", sklearn.preprocessing.StandardScaler() , [26]),  
]);

In [4]:
def fit_knn(x, y, params):
  # implementa knn con key word args
    clf = GridSearchCV(KNeighborsClassifier(), params, scoring='f1_macro')
    return clf.fit(x, y)

In [5]:
def eval_classifier(classifier, x_valid, y_valid, y_pred):
  # nos da las medidas recall, precision, f1 y una matriz de confusión. En principio vale para cualquier clasificador
    print(classification_report(y_pred, y_valid))
    plot_confusion_matrix(classifier, x_valid, y_valid, display_labels=set(y_valid.values), cmap=plt.cm.Blues)

In [6]:
#provisional: dejar sólo datos numéricos
x, y = load_data('train')
x = pd.DataFrame(column_transformerX.fit_transform(x))
x_valid, y_valid = load_data('validation')
x_valid = pd.DataFrame(column_transformerX.fit_transform(x_valid))

In [9]:
knn_params = {
    "n_neighbors": list(range(4, 8)),
    "weights": ['uniform', 'distance'],
}

In [ ]:
# se llama a la función de knn usando keyword args (podemos hacer una función que instancie varios classifiers con distintos parámetros)
classifier_1 = fit_knn(x, y, knn_params)

In [ ]:
classifier_1.best_params_

In [ ]:
y_pred = classifier_1.predict(x_valid)
eval_classifier(classifier_1, x_valid, y_valid, y_pred) 

In [ ]:
x_test = load_data('test')
x_ids = x_test['ID'].values
x_test = pd.DataFrame(column_transformerX.fit_transform(x_test))
pred = classifier_1.predict(x_test)
results = pd.DataFrame(list(zip(x_ids, pred)), columns=['ID', 'Credit_Score'])

In [ ]:
results.to_csv('predictions.csv', index=False)